# Data Dashboard and Storytelling Performance Assessment

### John Foster
### Department of Information Technology, Western Governors University
### D210: Representation and Reporting
### Professor Daniel Smith
### May 22, 2023

### A: INTERACTIVE DATA DASHBOARD

My Tableau dashboard has been created using Tableau Public and openly published via that service. The dashboard can be accessed by following this link: “https://public.tableau.com/app/profile/john.foster2076/viz/BroadbandServiceandTabletOwnershipMetrics/Dashboard1?publish=yes” The decision to use Tabeau Public was made to ensure that access to the dashboard could be freely accessed regardless of whether the viewer has access to a Tableau license. For anyone who wants to alter or explore the source workbook in more detail, it can also be found and downloaded freely via that link.

### A1: DATA SETS

The dashboard and associated workbook were constructed using 2 datasets: the provided WGU MSDA “Churn” dataset, and “S2801: Types of Computers and Internet Subscriptions,” published by the United States Census Bureau, hereafter referred to as the “Census” dataset (American Community Survey, 2021). The Churn dataset has been imported to the Tableau workbook unmodified, but the Census dataset had the data points relevant to our storytelling and relevant metrics extracted and used to generate an equivalent dummy dataset to Churn for comparison and cross-referencing.

The code and details of that data processing is located and annotated in the code snippets included in this notebook. The successful execution of this code will be demonstrated in a secondary Panopto presentation included with the submission, and is distinct from the primary Panopto presentation described in the PA rubric.

In [22]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import warnings

#ignore extraneous warnigns
warnings.filterwarnings('ignore')

#Import the Census dataset from the source csv file and make a dataframe out of it.
raw_df = pd.read_csv('ACSST1Y2021.S2801-2023-05-14T010706.csv')

#Make a copy of the imported dataframe in case we would like to inspect or manipulate the raw data. 
dfc = raw_df.copy()

#Storing tablet ownership percentage
df_tablet = dfc['United States!!Percent!!Estimate'][dfc['Label (Grouping)'] == '            Tablet or other portable wireless computer']
df_tablet = float(str(df_tablet).replace('%','').split(" ")[4].split("\n")[0])

df_tablet_count = int(df_tablet*100)

#Storing row indexing for income brackets
df_low_row = dfc[dfc['Label (Grouping)'] == '        Less than $20,000:'].index[0]
df_mid_row = dfc[dfc['Label (Grouping)'] == '        $20,000 to $74,999:'].index[0]
df_high_row = dfc[dfc['Label (Grouping)'] == '        $75,000 or more:'].index[0]

#Count of surveyed households by income brackets
low_count = int(dfc['United States!!Total!!Estimate'].iloc[df_low_row].replace(',',''))
mid_count = int(dfc['United States!!Total!!Estimate'].iloc[df_mid_row].replace(',',''))
high_count = int(dfc['United States!!Total!!Estimate'].iloc[df_high_row].replace(',',''))

#Storing total count for calculations
total_count = low_count + mid_count + high_count

#Storing bracket percentages for dummy data generation
low_percent = round((low_count / total_count), 3)
mid_percent = round((mid_count / total_count), 3)
high_percent = round((high_count / total_count), 3)

#String manipulation to format our cell values so that they can be processed as integers
low_BB_percent = float(dfc['United States!!Percent!!Estimate'].iloc[df_low_row+2].replace('%',''))
mid_BB_percent = float(dfc['United States!!Percent!!Estimate'].iloc[df_mid_row+2].replace('%',''))
high_BB_percent = float(dfc['United States!!Percent!!Estimate'].iloc[df_high_row+2].replace('%',''))

Tablet_series = pd.Series()

#Add 6380 values of 'Yes' to Tablet_series, and 3620 values of 'No' to adhere to our identified percentages
Tablet_series = Tablet_series.append(pd.Series(['Yes'] * df_tablet_count))
Tablet_series = Tablet_series.append(pd.Series(['No'] * (10000-df_tablet_count)))

#shuffle the rows in Tablet_series with a random state of 0
Tablet_series = Tablet_series.sample(frac=1, random_state=0).reset_index(drop=True)

#Making a blank column
IncomeBin_df = pd.DataFrame()
IncomeBin_df['IncomeBin'] = ''

#Add 6380 values of 'Yes' to Tablet_series, and 3620 values of 'No'
IncomeBin_Series = pd.Series()

#Appending our dummy income bracket values
IncomeBin_Series = IncomeBin_Series.append(pd.Series(['0-19,999'] * round(low_percent*10000)))
IncomeBin_Series = IncomeBin_Series.append(pd.Series(['20,000-74,999'] * round(mid_percent*10000)))
IncomeBin_Series = IncomeBin_Series.append(pd.Series(['75,000 and above'] * round(high_percent*10000)))

IncomeBin_df['IncomeBin'] = IncomeBin_Series.copy()

low_indices = []
mid_indices = []
high_indices = []

#Return the indices (row numbers) for each income bracket so that we can appropriately distribute our Broadband values within them
for i in range(len(IncomeBin_df)):
    if IncomeBin_df['IncomeBin'].iloc[i] == '0-19,999':
        low_indices.append(i)
    elif IncomeBin_df['IncomeBin'].iloc[i] == '20,000-74,999':
        mid_indices.append(i)
    elif IncomeBin_df['IncomeBin'].iloc[i] == '75,000 and above':
        high_indices.append(i)

#Copy our bracket values and prepare a column for Broadband with null values
IncomeBin_df['IncomeBin'] = IncomeBin_df['IncomeBin'].copy()
IncomeBin_df['Broadband'] = np.nan
    
#Programmatically distribute our Broadband "Yes" values within the row indices of the given income bracket
for i in range(int(len(low_indices) * (low_BB_percent/100))):
    IncomeBin_df['Broadband'].iloc[low_indices[i]] = 'Yes'
for i in range(int(len(mid_indices) * (mid_BB_percent/100))):
    IncomeBin_df['Broadband'].iloc[mid_indices[i]] = 'Yes'
for i in range(int(len(high_indices) * (high_BB_percent/100))):
    IncomeBin_df['Broadband'].iloc[high_indices[i]] = 'Yes'

#Fill our null spaces with "No" values and shuffle our indices so that they are randomly distributed
IncomeBin_df = IncomeBin_df.fillna('No')
IncomeBin_df = IncomeBin_df.sample(frac=1, random_state=0).reset_index(drop=True)
IncomeBin_df['CensusTablet'] = Tablet_series.copy()

#Generate a dummy "CaseOrder" column so our dataframe can be related to our Churn dataset using the same column
IncomeBin_df = IncomeBin_df.rename_axis('CaseOrder').reset_index()
IncomeBin_df['CaseOrder'] = IncomeBin_df['CaseOrder'] + 1

#Count the number of rows with '0-19,999' in the IncomeBin column and 'Yes' in the Broadband column
low_BB_count = len(IncomeBin_df[(IncomeBin_df['IncomeBin'] == '0-19,999') & (IncomeBin_df['Broadband'] == 'No')])
low_income_count = len(IncomeBin_df[(IncomeBin_df['IncomeBin'] == '0-19,999')])
mid_income_count = len(IncomeBin_df[(IncomeBin_df['IncomeBin'] == '20,000-74,999')])
#print(low_income_count)

xlow = 17233675
xmid = 50720161
xhigh = 59590894
xtotal = xlow+xmid+xhigh
xlowp = xlow/xtotal
xmidp = xmid/xtotal
xhighp = xhigh/xtotal
xtotalp = xlowp+xmidp+xhighp

#print(xlowp)
#print(xmidp)
#print(xhighp)
#print(xtotalp)

#print(low_percent)
#		--FROM generate_series(1, (SELECT ROUND(((income_count_low/income_count_total)*(broadband_percent_low/100))*10000)))
xly = int((xlow/xtotal) * (low_BB_percent/100) * 10000)
xln = int((xlow/xtotal) - ((xlow/xtotal) * (low_BB_percent/100) * 10000))

xmy = int((xmid/xtotal) * (mid_BB_percent/100) * 10000)
xmn = int((xmid/xtotal) - ((xmid/xtotal) * (mid_BB_percent/100) * 10000))

xhy = int((xhigh/xtotal) * (high_BB_percent/100) * 10000)
xhn = int((xhigh/xtotal) - ((xhigh/xtotal) * (high_BB_percent/100) * 10000))

xt = xly + xln + xmy + xmn + xhy + xhn

print(xly)
print(xln)
print(xmy)
print(xmn)
print(xhy)
print(xhn)


#Export the dataset
IncomeBin_df.to_csv('Census.csv', index=False)

994


### A2: INSTALLATION INSTRUCTIONS

Because the dashboard has been published on Tableau Public, it requires no installation to view and explore. It can be accessed via this link “https://public.tableau.com/app/profile/john.foster2076/viz/BroadbandServiceandTabletOwnershipMetrics/Dashboard1?publish=yes” and will appear via the Tableau Webapp in the user’s browser. 

### A3: NAVIGATION INSTRUCTIONS

The dashboard requires minimal navigation, and is contained in a single tab composed of 4 visualizations. Regardless of parameter settings, the left half of the dashboard represents data that is purely confined to the Churn dataset. When the Customer Data Parameter drop down is set to “Charts,”  the top left quadrant contains a stacked bar chart visualizing the percentages of customers who do or do not subscribe to the organization’s broadband internet services. The customer base is divided into 3 bars representing income brackets in this visualization. The bottom left quadrant contains a pie chart that represents the percentage of customers who own or do not own a tablet device. When the “Customer Parameter” drop down control is changed from “Charts” to “Maps,” these two visualizations are replaced with other visualizations. In the top left quadrant, the alternate visualization is a map with data points corresponding to generated geolocational data corresponding with each customer’s zip code. Each data point is color coded to represent the income bracket of the customer that it corresponds to. In the bottom left quadrant, the pie chart is replaced by a map of the United States with each state shaded differently. The intensity of the shading corresponds to the percentage of total customers in that state who own a tablet device.

Regardless of parameter settings, the right half of the dashboard represents data either exclusively sourced from the Census dataset, or generated via a comparison between the customer data contained in the Churn Dataset, and the corresponding measures extracted from the Census dataset. When the Census Data Parameter dropdown is set to “Charts,” the top right quadrant contains a stacked bar chart visualizing the percentages of American households who do or do not have broadband internet service. The households are divided into 3 bars representing income brackets in this visualization. It is formatted identically to its equivalent on the “Customer” half of the dashboard. When the Census Data Parameter dropdown is set to “Charts,” the bottom right quadrant contains a pie chart that represents the percentage of American households who own or do not own a tablet device. When the Census Data Parameter is changed from “Charts” to “Difference Tables,” the top right visualization is replaced with a table representing the percentage difference between the percentage of customers who purchase broadband service from the organization, and the total percentage of American households who have broadband service relative to that income bracket. In the lower right quadrant, the pie chart is replaced by a chart that represents the percentage difference between the percentage of organization customers who own a tablet, and the total percentage of American households that own a tablet.

The map visualizations can both be interacted with in a variety of ways. The scroll wheel can be used to zoom in and out, where holding shift while clicking and dragging on the map will allow the user to pan the view to different regions. However, the simplest way to focus the map view is to click the magnifying glass icon and enter text in the search bar, such as a city or state name. This will automatically focus the map on a given geographic area.

### B: PANOPTO STORYTELLING WITH DATA

The primary Panopto presentation can be found via the following link: “https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=e097173b-2060-4955-88bd-b00b0014cc10” This link will also be submitted along with the rest of the required materials for evaluation via the WGU submission portal. In addition to the submission link, I will also submit the powerpoint presentation featured in the Panopto video.

### C1: DASHBOARD ALIGNMENT

This dashboard explores two aspects related to the provided data dictionary’s expressed goal of customer retention and churn mitigation. 

First, it explores how our percentages of customers purchasing broadband internet services compare to national averages at different income thresholds to determine which populations are not being effectively engaged with or marketed to, the degree to which the customer base is being underserved, and even a means to visualize the how those economic groups are distributed and localized in different regions, thereby enabling more targeted marketing efforts.

Second, it explores how our percentages of customers in ownership of a tablet device compares to the national average, the degree to which we differ from that average, and geolocational insights outlining which states demonstrate the highest or lowest relative percentages of tablet ownership for targeted marketing and strategic purposes.

These insights enable executive leadership to more effectively market to and engage with demographically targeted and localized customer bases, which will ultimately serve the overall goal of mitigating customer churn.

### C2: ADDITIONAL DATA SET INSIGHTS

The variables extracted from the Census dataset are critical in contextualizing the customer data contained in the Churn dataset. It is relatively simple to evaluate our customer base across different services, and even further break those percentages down across different demographic features, but the resulting figures are meaningless without any context of where they should or should not fall. Having the ability to make direct comparisons between a customer base and the greater American population gives executive leadership insight into how well or poorly the organization is doing along that service axis. As a telecom company, our broadband service numbers should logically be higher than the national average, as our customers are ostensibly seeking out our services for the sole purpose of meeting those needs. With regards to tablet ownership, a lower percentage of customers who own tablets relative to the national average might indicate a marketing opportunity for device sales or inclusion of financed tablet devices as a means to drive traffic to customers signing one-year or two-year service contracts, thereby maximizing their tenure.

### C3: DECISION-MAKING SUPPORT

The two data representation that form the basis for executive decision making across the metrics of broadband service vs. income bracket and tablet ownership are the customer income bracket symbol map (contained in the top left quadrant with the Customer Data Parameter dropdown set to “Maps”), and the state map containing relative tablet ownership percentages (contained in the bottom left quadrant with the Customer Data Parameter dropdown set to “Maps”). While the percentage difference tables outline the explicit need for action, the map features allow executives to begin strategizing localized efforts to cater to the identified segments of the customer base. This ultimately assists leadership in rectifying the discrepancies between our customer base and the national averages while also driving further targeted data collection in order to better understand the factors underpinning regional deviations. In effect, it enables executives to prioritize focus on the regions most in need of attention with regards to our identified metrics.

### C4: INTERACTIVE CONTROLS

As described above, the interactive controls of the dashboard take the form of two drop down parameter selection controls. 

The dropdown on the left half of the screen allows the viewer to swap between charts of our customer data relative to the two identified metrics (by selecting the “Charts” option), and geolocational visualizations illustrating regional concentrations of customers across income brackets, and relative tablet ownership by state respectively (by selecting the “Maps” option).

On the right half of the screen, the dropdown allows the viewer to swap between charts of national Census averages relative to our two identified metrics (by selecting the “Charts” option), and tables containing the percentage differences between our customer metric percentages, and national percentages relative to both metrics. Dropdown controls were developed using instructions adapted from the "OneNumber - Tableau Experts" Youtube Channel (2022) via the following link: "https://www.youtube.com/watch?v=fExopuDJz7g"

### C5: COLORBLINDNESS

To make the presentation accessible to users affected by all different types of colorblindness, I elected to render the dashboard entirely in grayscale. This should also accommodate users who experience other visual impairments, as the layout is naturally high in contrast.

### C6: DATA REPRESENTATIONS

The two data representations most critical in the storytelling of this analysis are the percent difference tables displayed by setting the Census Data Parameter dropdown box to “Difference Tables”. These tables function as representations of the two key performance indicators explored in this analysis. The first table succinctly summarizes how well we are marketing our broadband internet services to different income groups in our customer base relative to national rates of broadband access. The second table demonstrates the extremely low rate of tablet ownership among our customers, clearly identifying a potential market within our customer base in need of devices and opening the door to the upselling of more profitable service add-ons while getting tablet devices into customers’ hands.

### C7: AUDIENCE ANALYSIS

The dashboard has been built as simply as possible, with no tabs to navigate and minimal interactive controls, in order for executive leadership to quickly and effectively absorb the story of our two selected service metrics. With regards to the presentation itself, multiple talking points were chosen to speak directly to the interests of specific audience members. Those talking points will be outlined below.

The Senior Vice President for Customer Experience (SVP) is, according to the provided data dictionary, primarily focused on customer engagement and retention, and is principally focused on key characteristics of given customer groups which might drive behavior. In the presentation, I tried to speak directly to the SVP in my recommendations for incentivized customer data collection in the form of surveying to understand the behavioral patterns that underpin the discrepancies between our numbers and the national averages across both metrics. I also had the SVP in mind when recommending focused, granular analyses of that survey data against customer demographic features.

The Executive Vice President of Sales (EVP) has a slightly different role, being tasked with recruitment, retention, renewal sales, and planning the development of promotions and products to reach broad categories of customers. The EVP was who I specifically had in mind when choosing and exploring my two metrics. For broadband service across income levels, I spoke in the presentation to the specific income groups that we appear to be missing in our delivery of those services, how that might also translate to potential new customers within those income levels. The inclusion of a symbol map of customers’ income levels also spoke to the EVP’s interest in regional demographic breakdowns. In our metric of tablet ownership, I appealed to the EVP by proposing the inclusion of discounted or financed tablet devices as potential customer incentives in service contracts, maximizing retention and creating targeted service upselling opportunities.

The Panel of Regional Vice Presidents (Regional VP) are chiefly concerned with actionable insights that will drive their own regional policies, initiatives, and operational goals. The regional concentrations of income brackets in the symbol map and the map of tablet ownership by state gives regional VP’s an immediate indication as to the priority of developing regionally appropriate initiatives relative to these metrics within their own sectors. For example, the Regional VP of a region with lower concentrations of middle income and high income customers might first choose to focus on operational strategies related to tablet ownership if the states in their region demonstrate low percentages in that metric. In effect, the maps allow them an immediate point of comparison to other regions and illustrate how affected their region with reference to these metrics. Within the presentation, the included explanation of how to read and interact with the maps was primarily included for the benefit of Regional VP’s exploring these metrics.

My peers on the Data Analytics team will be primarily focused on the underlying story and mechanics of this analysis can be leveraged in further research in order to expand upon the insights offered. In my presentation, I specifically targeted my teammates by detailing what doors the primary metrics derived from this analysis (namely, the relative differences between our own numbers and national averages) open with regards to further research and analysis. I specifically outlined several analyses that could immediately be planned in order to generate a more granular understanding of how our two metrics impact different regions and demographics. The data-driven story told in this analysis is short and fairly straightforward to summarize: Middle income and high income households are not purchasing broadband services from us at levels we would expect, and our customer base has a conspicuously low rate of tablet ownership. These are clear insights, but without further analysis as to their underlying correlations with other features, any actions taken to rectify them will be largely undertaken with a certain degree of contextual blindness. Outlining the next steps for my peers enables this analysis to contribute to a much larger and more comprehensive data-driven story related to these metrics.

### C8: UNIVERSAL ACCESS

In addition to making the presentation in grayscale to cater to users affected by colorblindness, the dashboard and presentation have all been designed with textual representations of the data being described so that software designed to assist individuals with other visual impairments can derive the same insights from the data presented. The Panopto presentation also includes generated captions to assist viewers with hearing impairments. Publishing the dashboard on Tableau public also guarantees universal access to users across different platforms and operating systems by delivering the dashboard via a web application in the user’s browser.

### C9: EFFECTIVE STORYTELLING

Two elements of effective storytelling that I used to engage my audience were a contextual explanation of the datasets in context, and a comprehensive breakdown of actionable insights relevant to the audience present (Kamara, 2021).

Establishing a contextual understanding of the datasets used in this analysis is critical in communicating what the figures and measures presented actually mean. Explaining the features our internal dataset is fairly straightforward, but since the income columns and Internet Service columns in particular were processed to make them correspond more directly to the features contained in the Census dataset, explaining that process early in the presentation was very important in justifying the comparisons made between our customer base and national averages. With respect to the census data itself, I provided a brief explanation of how I generated the dummy dataset so that it would be directly comparable to our own internal data. These explanations were delivered as simply as possible in order to maintain the pace of the presentation.

The actionable insights were more difficult to organize for the benefit of the different sections of my audience. Instead of dividing those insights into sections relevant to each audience portion, I instead tried to seed them evenly between my explanations of each metric in an attempt to seed the information and insight most relevant to each audience portion as close to equidistant as possible throughout the presentation. The goal of structuring the presentation this way was to prioritize sustained engagement of each individual portion of the audience, and give as few opportunities as possible for different audience portions to become distracted.

### H: SOURCES

Kamara, K. (2021). D210 Performance Assessment Part 3: Reflection Paper. Western Governors University, College of Information Technology. https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=7675c6a3-aced-4a60-8bb7-ad9500ff3ab8.

American Community Survey. (2021). S2801: Types of Computers and Internet Subscriptions. United States Census Bureau. https://data.census.gov/table?q=broadband&tid=ACSST1Y2021.S2801.

### H1: THIRD PARTY TABLEAU INSTRUCTIONS

OneNumber - Tableau Experts. (2022). How to Swap Worksheets in a Tableau Dashboard (v 2022.2+). https://www.youtube.com/watch?v=fExopuDJz7g.